In [ ]:
import os.path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import constants
from IPython.display import display
import ipywidgets as widgets

def H1(text):
    return widgets.HTML(f"<h1>{text}</h1>")
    
def H2(text):
    return widgets.HTML(f"<h2>{text}</h2>")

def evaluate_all(data_dir):
    display(H1("Teil 3: Bestimmung von e/m"))
    
    e = constants.elementary_charge
    m_e = constants.electron_mass

    def B(I):
        return 0.78e-3 * I
    
    def em(U, B, R):
        return 2 * U / (B**2 * R**2)
    
    #def v(spez_elm):
    #    return np.sqrt(1 * 2 * spez_elm)
    
    data = np.load(os.path.join(data_dir, "DataV30Part1.npy"))

    df = pd.DataFrame({
        "voltage": data[0],
        "current": data[1],
        "diameter": data[2] * 1e-2,  # cm -> m
    })
    df["radius"] = df.diameter / 2 
    df["B"] = B(df.current)
    
    df["e_over_m"] = em(df.voltage, df.B, df.radius)
    df["e/m [C/kg]"] = np.round(df.e_over_m, -7)
    display(df)
    print("Mittelwert = {0:.3e} C/Kg".format(np.mean(df.e_over_m)))
    print("Literaturwert = {0:.3e} C/Kg".format(e / m_e))
    
    
    display(H1("Frage nach der Durchführung"))
    
    m_e_calc = e / np.mean(df.e_over_m)
    print("m_e calculated = %.3e kg" % m_e_calc)
    print("m (literature value) = %.3e kg" % m_e)

    v_lit = np.sqrt(2 * e * 1 / m_e)
    v_calc = np.sqrt(2 * np.mean(df.e_over_m) * 1)
    print("velocity with literature values = %.3e m/s" % v_lit)
    print("velocity with calculated e/m value = %.3e m/s" % v_calc)

# Interactive student selection
with open("students.txt", "r") as f:
    students = [line.strip("\n") for line in f if not line.startswith("#")]
    
tb = widgets.ToggleButtons(
    options=students,
    description='Name:',
)
display(tb)

output = widgets.Output()
display(output)

@output.capture(clear_output=True)
def on_student_change(change):
    name = change["new"]
    display(H1(f"Auswertung für: {name}"))

    name_escaped = name.replace(" ", "_")
    data_dir = os.path.join("V30", name_escaped)
    
    evaluate_all(data_dir)

tb.observe(on_student_change, names="value")
on_student_change({"new": students[0]})